<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim
#### Alumna: Ariadna Garmendia



### Enunciado
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. 


In [1]:
import json
import string
import random
import re # Regular Expressions (regex)
import urllib.request

import numpy as np

# Para leer y parsear texto HTML
import bs4 as bs

# Para evitar respuesta "forbidden"
import ssl

import warnings
from pprint import pprint
import requests

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec # Word2Vec es el generador de embeddings de Gensim

### Datos
Utilizo un cuento en expañol llamado "El caballero de la armadura oxidada"

In [3]:
#context = ssl._create_unverified_context()

user_agent = 'Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.9.0.7) Gecko/2009021910 Firefox/3.0.7'

url = "http://www.librosmaravillosos.com/elcaballerodelaarmaduraoxidada/index.html"

headers={'User-Agent':user_agent,} 

request=urllib.request.Request(url,None,headers) #The assembled request
response = urllib.request.urlopen(request)
raw_html = response.read() # The raw page

# Parsear artículo, 'lxml' es el parser a utilizar
article_html = bs.BeautifulSoup(raw_html, 'lxml')

# Encontrar todos los párrafos del HTML (bajo el tag <p>)
# y tenerlos disponible como lista
article_paragraphs = article_html.find_all()

article_text = ''

for para in article_paragraphs:
    article_text += para.text

# Elimino lo que no me interesa
separator_end = 'F I NF I N' # Separador a partir del cual remuevo todo lo que sigue
corpus = article_text.split(separator_end, 1)[0] 
corpus = corpus[corpus.rfind('Hace ya mucho tiempo,'):] # Remuevo lo que está antes de esta frase
corpus = re.sub('Capítulo 2En los bosques de Merlín', ' ', corpus)
corpus = re.sub('Capítulo 3El sendero de la verdad', ' ', corpus)
corpus = re.sub('Capítulo 4El castillo del silencio', ' ', corpus)
corpus = re.sub('Capítulo 5El castillo del conocimiento', ' ', corpus)
corpus = re.sub('Capítulo 6El castillo de la voluntad y la osadía', ' ', corpus)
corpus = re.sub('Capítulo 7La cima de la verdad', ' ', corpus)
corpus = re.sub(r'—', ' ', corpus) # 
corpus = re.sub(r'\s+', ' ', corpus) # substituir más de un caracter de espacio, salto de línea o tabulación



In [4]:
corpus

'Hace ya mucho tiempo, en una tierra muy lejana, vivía un caballero que pensaba que era bueno, generoso y amoroso. Hacía todo lo que suelen hacer los caballeros buenos, generosos y amorosos. Luchaba contra sus enemigos, que era malos, mezquinos y odiosos. Mataba a dragones y rescataba a damiselas en apuros. Cuando en el asunto de la caballería había crisis, tenía la mala costumbre de rescatar damiselas incluso cuando ellas no deseaban ser rescatadas y, debido a esto, aunque muchas damas le estaban agradecidas, otras tantas se mostraban furiosas con el caballero. Él lo aceptaba con filosofía. Después de todo, no se puede contentar a todo el mundo.Nuestro caballero era famoso por su armadura. Reflejaba unos rayos de luz tan brillantes que la gente del pueblo juraba no haber visto el sol salir en el norte o ponerse en el este cuando el caballero partía a la batalla. Y partía a la batalla con bastante frecuencia. Ante la mera mención de una cruzada, el caballero se ponía la armadura entusi

In [5]:
# Separo las frases usando el delimitador que usé (.)
corpus = corpus.split('.') 

In [6]:
corpus[0:25]

['Hace ya mucho tiempo, en una tierra muy lejana, vivía un caballero que pensaba que era bueno, generoso y amoroso',
 ' Hacía todo lo que suelen hacer los caballeros buenos, generosos y amorosos',
 ' Luchaba contra sus enemigos, que era malos, mezquinos y odiosos',
 ' Mataba a dragones y rescataba a damiselas en apuros',
 ' Cuando en el asunto de la caballería había crisis, tenía la mala costumbre de rescatar damiselas incluso cuando ellas no deseaban ser rescatadas y, debido a esto, aunque muchas damas le estaban agradecidas, otras tantas se mostraban furiosas con el caballero',
 ' Él lo aceptaba con filosofía',
 ' Después de todo, no se puede contentar a todo el mundo',
 'Nuestro caballero era famoso por su armadura',
 ' Reflejaba unos rayos de luz tan brillantes que la gente del pueblo juraba no haber visto el sol salir en el norte o ponerse en el este cuando el caballero partía a la batalla',
 ' Y partía a la batalla con bastante frecuencia',
 ' Ante la mera mención de una cruzada,

In [7]:
len(corpus)

1121

In [8]:
df = pd.DataFrame(corpus)

In [9]:
# Armar el dataset 
df.head(10)

,0
0,"Hace ya mucho tiempo, en una tierra muy lejana..."
1,Hacía todo lo que suelen hacer los caballeros...
2,"Luchaba contra sus enemigos, que era malos, m..."
3,Mataba a dragones y rescataba a damiselas en ...
4,Cuando en el asunto de la caballería había cr...
5,Él lo aceptaba con filosofía
6,"Después de todo, no se puede contentar a todo..."
7,Nuestro caballero era famoso por su armadura
8,Reflejaba unos rayos de luz tan brillantes qu...
9,Y partía a la batalla con bastante frecuencia


In [10]:
print("Cantidad de documentos:", df.shape[0])

Cantidad de documentos: 1121


In [11]:
df.empty

False

### 1 - Preprocesamiento

####  1.1 Dividir los documentos en palabras (tokenizar) - Uso tokenizador de Keras

In [12]:
from keras.preprocessing.text import text_to_word_sequence

sentence_tokens = []
# Recorrer todas las filas y transformar las oraciones
# en una secuencia de palabras (esto podría realizarse con NLTK o spaCy también)
for _, row in df[:None].iterrows():
    sentence_tokens.append(text_to_word_sequence(row[0])) # Acá guardo todos los tokens

In [13]:
# Miro como quedaron las primeras 5 oraciones
sentence_tokens[:5]

[['hace',
  'ya',
  'mucho',
  'tiempo',
  'en',
  'una',
  'tierra',
  'muy',
  'lejana',
  'vivía',
  'un',
  'caballero',
  'que',
  'pensaba',
  'que',
  'era',
  'bueno',
  'generoso',
  'y',
  'amoroso'],
 ['hacía',
  'todo',
  'lo',
  'que',
  'suelen',
  'hacer',
  'los',
  'caballeros',
  'buenos',
  'generosos',
  'y',
  'amorosos'],
 ['luchaba',
  'contra',
  'sus',
  'enemigos',
  'que',
  'era',
  'malos',
  'mezquinos',
  'y',
  'odiosos'],
 ['mataba',
  'a',
  'dragones',
  'y',
  'rescataba',
  'a',
  'damiselas',
  'en',
  'apuros'],
 ['cuando',
  'en',
  'el',
  'asunto',
  'de',
  'la',
  'caballería',
  'había',
  'crisis',
  'tenía',
  'la',
  'mala',
  'costumbre',
  'de',
  'rescatar',
  'damiselas',
  'incluso',
  'cuando',
  'ellas',
  'no',
  'deseaban',
  'ser',
  'rescatadas',
  'y',
  'debido',
  'a',
  'esto',
  'aunque',
  'muchas',
  'damas',
  'le',
  'estaban',
  'agradecidas',
  'otras',
  'tantas',
  'se',
  'mostraban',
  'furiosas',
  'con',
  'el'

### 2 - Crear los vectores (word2vec)

#### 2.1 Callback para graficar loss a medida que se entrena Gensim

In [14]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobracargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

#### 2.2 Defino los parámetros del modelo

MODELO NRO 1 A PROBAR (SKIPGRAM):

In [15]:
# Crearmos el modelo generador de vectoeres
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario (si aparece menos de 5 veces, Gensim la descarta)
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=50,       # dimensionalidad de los vectores (de salida)
                     negative=20,    # cantidad de negative samples (las más representativas)... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

#### 2.3 Armo el vocabulario

MODELO NRO 1

In [16]:
# Buildear el vocabulario con los tokens
w2v_model.build_vocab(sentence_tokens)

In [17]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model.corpus_count)

Cantidad de docs en el corpus: 1121


In [18]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model.wv.vocab))

Cantidad de words distintas en el corpus: 455


In [19]:
w2v_model.wv.vocab

{'hace': <gensim.models.keyedvectors.Vocab at 0x7f734e80fc10>,
 'ya': <gensim.models.keyedvectors.Vocab at 0x7f734e80fc50>,
 'mucho': <gensim.models.keyedvectors.Vocab at 0x7f734e80fc90>,
 'tiempo': <gensim.models.keyedvectors.Vocab at 0x7f734e80fcd0>,
 'en': <gensim.models.keyedvectors.Vocab at 0x7f734e80fd50>,
 'una': <gensim.models.keyedvectors.Vocab at 0x7f734e80fdd0>,
 'muy': <gensim.models.keyedvectors.Vocab at 0x7f734e80fe10>,
 'lejana': <gensim.models.keyedvectors.Vocab at 0x7f734e80fe50>,
 'un': <gensim.models.keyedvectors.Vocab at 0x7f734e80fd10>,
 'caballero': <gensim.models.keyedvectors.Vocab at 0x7f734e80fd90>,
 'que': <gensim.models.keyedvectors.Vocab at 0x7f734e80fe90>,
 'pensaba': <gensim.models.keyedvectors.Vocab at 0x7f734e80fed0>,
 'era': <gensim.models.keyedvectors.Vocab at 0x7f734e80ff10>,
 'bueno': <gensim.models.keyedvectors.Vocab at 0x7f734e80ff50>,
 'generoso': <gensim.models.keyedvectors.Vocab at 0x7f734e80ff90>,
 'y': <gensim.models.keyedvectors.Vocab at 0x7f

#### Nota: Es importante verificar la relación entre la cantidad de palabras distintas y la cantidad de documentos (si no tengo suficientes documentos, tendría que considerar conseguir más datos). En este caso la relación no es demasiado grande.

### 3 - Entrenar el modelo generador

In [20]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model.train(sentence_tokens,
                 total_examples=w2v_model.corpus_count,
                 epochs=2000,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 131642.609375
Loss after epoch 1: 69173.484375
Loss after epoch 2: 68818.5
Loss after epoch 3: 68297.25
Loss after epoch 4: 69722.875
Loss after epoch 5: 68597.53125
Loss after epoch 6: 67334.6875
Loss after epoch 7: 69152.5
Loss after epoch 8: 70996.5
Loss after epoch 9: 68548.625
Loss after epoch 10: 68871.75
Loss after epoch 11: 67932.8125
Loss after epoch 12: 67133.0625
Loss after epoch 13: 66634.4375
Loss after epoch 14: 61493.625
Loss after epoch 15: 58107.25
Loss after epoch 16: 57744.875
Loss after epoch 17: 56499.125
Loss after epoch 18: 56224.5
Loss after epoch 19: 54786.25
Loss after epoch 20: 55461.0
Loss after epoch 21: 54022.625
Loss after epoch 22: 54549.75
Loss after epoch 23: 53794.75
Loss after epoch 24: 52945.5
Loss after epoch 25: 52298.75
Loss after epoch 26: 52276.375
Loss after epoch 27: 51107.75
Loss after epoch 28: 50929.5
Loss after epoch 29: 50799.5
Loss after epoch 30: 50701.375
Loss after epoch 31: 50447.25
Loss after epoch 32: 49395.625

(14172892, 31558000)

### 4 - Ensayar

a) Palabras que MÁS se relacionan con:

In [21]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["armadura"], topn=15)

[('desconocido', 0.4206427335739136),
 ('dormir', 0.40307942032814026),
 ('las', 0.3775336444377899),
 ('hijo', 0.37557804584503174),
 ('perdido', 0.3592827320098877),
 ('por', 0.35831478238105774),
 ('su', 0.3566811680793762),
 ('hablando', 0.34511977434158325),
 ('cómo', 0.3413276970386505),
 ('se', 0.33921492099761963),
 ('antes', 0.33032849431037903),
 ('dar', 0.32867902517318726),
 ('que', 0.32863420248031616),
 ('visto', 0.32785165309906006),
 ('de', 0.32713747024536133)]

In [22]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["caballero"], topn=25)

[('el', 0.9100741744041443),
 ('y', 0.7135035395622253),
 ('se', 0.697830080986023),
 ('que', 0.691400945186615),
 ('con', 0.6633176803588867),
 ('no', 0.6305489540100098),
 ('de', 0.6275771260261536),
 ('la', 0.5857557654380798),
 ('en', 0.5838313102722168),
 ('lo', 0.5596433281898499),
 ('una', 0.5123971104621887),
 ('le', 0.49254947900772095),
 ('pero', 0.47723114490509033),
 ('a', 0.4770013093948364),
 ('un', 0.4724651575088501),
 ('merlín', 0.46773457527160645),
 ('del', 0.46628692746162415),
 ('su', 0.4240797460079193),
 ('más', 0.4204378128051758),
 ('por', 0.40873292088508606),
 ('reflexionó', 0.3988945186138153),
 ('al', 0.3885550796985626),
 ('las', 0.3833645284175873),
 ('dragón', 0.34439218044281006),
 ('levadizo', 0.33261507749557495)]

In [24]:
w2v_model.wv.most_similar(positive=["Julieta"], topn=25)

KeyError: ignored

In [26]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["julieta"], topn=15)

[('bueno', 0.5286751389503479),
 ('cristóbal', 0.464471697807312),
 ('amaba', 0.4467121362686157),
 ('aquello', 0.4390237033367157),
 ('has', 0.4186362326145172),
 ('amado', 0.41749903559684753),
 ('parte', 0.41709965467453003),
 ('habían', 0.4003604054450989),
 ('había', 0.3948251008987427),
 ('comenzó', 0.3940368890762329),
 ('aunque', 0.3743031620979309),
 ('generoso', 0.36302709579467773),
 ('roca', 0.3550640642642975),
 ('porque', 0.35332396626472473),
 ('damiselas', 0.3496323823928833)]

In [27]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["ardilla"], topn=15)

[('bolsalegre', 0.5076074004173279),
 ('voluntad', 0.5027350187301636),
 ('rebeca', 0.45809024572372437),
 ('animales', 0.4499671459197998),
 ('movió', 0.42289525270462036),
 ('cola', 0.41460537910461426),
 ('has', 0.3995417058467865),
 ('merlín', 0.39647236466407776),
 ('corriendo', 0.3798247277736664),
 ('hablando', 0.3734656572341919),
 ('habéis', 0.3685861825942993),
 ('mientras', 0.36806005239486694),
 ('profundamente', 0.3672880232334137),
 ('durante', 0.34975317120552063),
 ('estas', 0.3477833867073059)]

In [28]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["paloma"], topn=10)

[('veces', 0.47220340371131897),
 ('misma', 0.4371826648712158),
 ('tengo', 0.42593175172805786),
 ('abrió', 0.42142271995544434),
 ('yelmo', 0.4200170636177063),
 ('mientras', 0.41435688734054565),
 ('¡no', 0.40630313754081726),
 ('batalla', 0.4023904502391815),
 ('decir', 0.4005279541015625),
 ('sería', 0.39540383219718933)]

In [29]:
w2v_model.wv.most_similar(positive=["rebeca"], topn=10)

[('ardilla', 0.45809024572372437),
 ('os', 0.4470615088939667),
 ('pasado', 0.40667232871055603),
 ('corriendo', 0.3913290202617645),
 ('regresar', 0.3882126808166504),
 ('coraje', 0.3814336657524109),
 ('les', 0.38104188442230225),
 ('poco', 0.37193557620048523),
 ('sería', 0.3710494637489319),
 ('caballo', 0.3606563210487366)]

In [30]:
# Palabras que MÁS se relacionan con...:
w2v_model.wv.most_similar(positive=["merlín"], topn=10)

[('que', 0.5417777299880981),
 ('no', 0.4886833429336548),
 ('a', 0.482021301984787),
 ('el', 0.4684999883174896),
 ('caballero', 0.46773457527160645),
 ('volvió', 0.4648396074771881),
 ('con', 0.44246166944503784),
 ('y', 0.4374358355998993),
 ('más', 0.4266934096813202),
 ('la', 0.42156243324279785)]

In [31]:
w2v_model.wv.most_similar(positive=["castillo"], topn=10)

[('silencio', 0.5889235734939575),
 ('conocimiento', 0.4494462013244629),
 ('algún', 0.39977821707725525),
 ('en', 0.3894762396812439),
 ('otro', 0.3894018828868866),
 ('elegante', 0.38670530915260315),
 ('habitación', 0.37799423933029175),
 ('siguiente', 0.3675212562084198),
 ('lejana', 0.36694976687431335),
 ('eres', 0.3629086911678314)]

In [32]:
w2v_model.wv.most_similar(positive=["miedo"], topn=15)

[('quiero', 0.4689388573169708),
 ('sorprendido', 0.4624476730823517),
 ('duda', 0.45101019740104675),
 ('osadía', 0.4323931932449341),
 ('pared', 0.3943994343280792),
 ('ninguna', 0.38416358828544617),
 ('conocimiento', 0.37872570753097534),
 ('voluntad', 0.3626060485839844),
 ('matar', 0.3616790771484375),
 ('uno', 0.3580966591835022),
 ('podéis', 0.35476362705230713),
 ('mejores', 0.35433998703956604),
 ('humor', 0.3541971743106842),
 ('rió', 0.353187620639801),
 ('primero', 0.33694350719451904)]

In [33]:
w2v_model.wv.most_similar(positive=["espada"], topn=15)

[('necesidad', 0.5047368407249451),
 ('bastante', 0.4999467730522156),
 ('sentía', 0.4881383776664734),
 ('amor', 0.46951523423194885),
 ('embargo', 0.46818917989730835),
 ('tal', 0.43259966373443604),
 ('cosas', 0.43148982524871826),
 ('mente', 0.4310183823108673),
 ('amado', 0.42093923687934875),
 ('mañana', 0.419707328081131),
 ('alguien', 0.41115665435791016),
 ('iba', 0.41035065054893494),
 ('mala', 0.4055016338825226),
 ('rostro', 0.4053732752799988),
 ('tendréis', 0.40151190757751465)]

In [34]:
w2v_model.wv.most_similar(positive=["rey"], topn=15)

[('podido', 0.46318283677101135),
 ('abrir', 0.4416574537754059),
 ('volver', 0.42374688386917114),
 ('algún', 0.4059978425502777),
 ('sonido', 0.4035577178001404),
 ('lado', 0.3973800539970398),
 ('bufón', 0.38295355439186096),
 ('listo', 0.3813322186470032),
 ('quiero', 0.3783131241798401),
 ('habéis', 0.36647385358810425),
 ('siempre', 0.36580580472946167),
 ('dolor', 0.35669922828674316),
 ('debe', 0.35661113262176514),
 ('antes', 0.35227739810943604),
 ('hasta', 0.3489934504032135)]

In [35]:
w2v_model.wv.most_similar(positive=["dragón"], topn=15)

[('puente', 0.5119661092758179),
 ('ellos', 0.41364559531211853),
 ('matar', 0.4121808409690857),
 ('culpa', 0.3969680666923523),
 ('oscuridad', 0.3915013074874878),
 ('demostrar', 0.3860476315021515),
 ('ser', 0.37833371758461),
 ('conocimiento', 0.37556883692741394),
 ('dragones', 0.37185269594192505),
 ('algo', 0.368404746055603),
 ('barba', 0.3663462698459625),
 ('gritó', 0.3631519675254822),
 ('sonido', 0.362311452627182),
 ('enorme', 0.3616534173488617),
 ('corazón', 0.3552550971508026)]

In [36]:
w2v_model.wv.most_similar(positive=["cima"], topn=15)

[('pie', 0.5558347105979919),
 ('ante', 0.4995821714401245),
 ('mala', 0.4658677279949188),
 ('montaña', 0.4478739798069),
 ('dio', 0.4461289346218109),
 ('casi', 0.4450017809867859),
 ('partir', 0.4394036531448364),
 ('puerta', 0.43754714727401733),
 ('verdad', 0.43472200632095337),
 ('llegar', 0.4296117126941681),
 ('entró', 0.426874577999115),
 ('mano', 0.4150473475456238),
 ('toda', 0.403726190328598),
 ('él', 0.3979846239089966),
 ('intentando', 0.39131104946136475)]

b) Palabras que MENOS se relacionan con:

In [55]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["armadura"], topn=10)

[('estás', 0.19449090957641602),
 ('preguntó', 0.18878579139709473),
 ('replicó', 0.18050797283649445),
 ('puente', 0.16912253201007843),
 ('apareció', 0.12834787368774414),
 ('levadizo', 0.12659820914268494),
 ('espejo', 0.12258513271808624),
 ('palabras', 0.11936280876398087),
 ('oído', 0.11345125734806061),
 ('desapareció', 0.11028817296028137)]

In [37]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["caballero"], topn=10)

[('menudo', 0.07658691704273224),
 ('tal', 0.048528701066970825),
 ('tanto', 0.043880559504032135),
 ('tendréis', 0.03867993876338005),
 ('debe', 0.03566401079297066),
 ('aún', 0.03362351655960083),
 ('viaje', 0.030147971585392952),
 ('intentando', 0.028880717232823372),
 ('atrapado', 0.02594609372317791),
 ('gente', 0.025839220732450485)]

In [38]:
# Palabras que MENOS se relacionan con...:
w2v_model.wv.most_similar(negative=["julieta"], topn=10)

[('gran', 0.1591128706932068),
 ('silencio', 0.14780810475349426),
 ('ha', 0.1434696465730667),
 ('cualquier', 0.136907160282135),
 ('parece', 0.13090702891349792),
 ('mago', 0.12591831386089325),
 ('mucho', 0.12521511316299438),
 ('nueces', 0.12454205006361008),
 ('sólo', 0.12194715440273285),
 ('reino', 0.10996676981449127)]

In [39]:
w2v_model.wv.most_similar(negative=["cristóbal"], topn=15)

[('donde', 0.2829255759716034),
 ('años', 0.22068147361278534),
 ('he', 0.19093184173107147),
 ('reino', 0.14993730187416077),
 ('estado', 0.14713265001773834),
 ('del', 0.10750675201416016),
 ('siquiera', 0.10437033325433731),
 ('hayáis', 0.1005595475435257),
 ('explicó', 0.08825932443141937),
 ('hecho', 0.07882867753505707),
 ('durante', 0.07732099294662476),
 ('e', 0.06898357719182968),
 ('siempre', 0.06534640491008759),
 ('silencio', 0.06443005800247192),
 ('acero', 0.06317701935768127)]

In [40]:
w2v_model.wv.most_similar(negative=["animales"], topn=15)

[('demostrar', 0.2156834751367569),
 ('toda', 0.19132380187511444),
 ('o', 0.1755378693342209),
 ('tenía', 0.12786075472831726),
 ('fuera', 0.12543225288391113),
 ('decir', 0.11582417786121368),
 ('¿qué', 0.10951532423496246),
 ('¿no', 0.09081094712018967),
 ('respondió', 0.08287039399147034),
 ('todo', 0.0800456702709198),
 ('comprender', 0.074195995926857),
 ('amor', 0.05940845236182213),
 ('parte', 0.058707669377326965),
 ('¿y', 0.05792577564716339),
 ('ella', 0.05751252919435501)]

### 5 - Visualizar agrupación de vectores con TSNE


In [41]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model):
    num_dimensions = 2  

    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index2word)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    x_vals = [v[0] for v in vectors]
    y_vals = [v[1] for v in vectors]
    return x_vals, y_vals, labels

In [42]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,



---



MODELO NRO 2 A PROBAR (CBOW):

a) Armo modelo

In [43]:
# Crearmos el modelo generador de vectores
# En este caso se usará CBOW
w2v_model_2 = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario (si aparece menos de 5 veces, Gensim la descarta)
                     window=2,       # cant de palabras antes y desp de la predicha
                     size=50,       # dimensionalidad de los vectores (de salida)
                     negative=20,    # cantidad de negative samples (las más representativas)... 0 es no se usa
                     workers=1,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [44]:
# Buildear el vocabulario con los tokens
w2v_model_2.build_vocab(sentence_tokens)

b) Entreno modelo

In [45]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_2.train(sentence_tokens,
                 total_examples=w2v_model_2.corpus_count,
                 epochs=2000,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 70685.984375
Loss after epoch 1: 32020.0625
Loss after epoch 2: 30317.09375
Loss after epoch 3: 29748.015625
Loss after epoch 4: 29990.703125
Loss after epoch 5: 29643.203125
Loss after epoch 6: 29218.203125
Loss after epoch 7: 29775.203125
Loss after epoch 8: 29868.40625
Loss after epoch 9: 29466.15625
Loss after epoch 10: 29637.1875
Loss after epoch 11: 29732.125
Loss after epoch 12: 29511.71875
Loss after epoch 13: 29564.4375
Loss after epoch 14: 29664.15625
Loss after epoch 15: 29317.8125
Loss after epoch 16: 29320.09375
Loss after epoch 17: 29017.3125
Loss after epoch 18: 28557.375
Loss after epoch 19: 27982.8125
Loss after epoch 20: 27979.1875
Loss after epoch 21: 27519.875
Loss after epoch 22: 27354.625
Loss after epoch 23: 26828.125
Loss after epoch 24: 26464.875
Loss after epoch 25: 25993.875
Loss after epoch 26: 25708.875
Loss after epoch 27: 25088.75
Loss after epoch 28: 24647.5625
Loss after epoch 29: 24376.5625
Loss after epoch 30: 24079.875
Loss after 

(14172892, 31558000)

d) Palabras que MÁS se parecen 

In [46]:
w2v_model_2.wv.most_similar(positive=["caballero"], topn=15)

[('el', 0.535047173500061),
 ('rey', 0.49865198135375977),
 ('dragón', 0.43063533306121826),
 ('mago', 0.38038501143455505),
 ('suelo', 0.3332987427711487),
 ('sorprendido', 0.3228212296962738),
 ('espejo', 0.3194670081138611),
 ('arriba', 0.3069528341293335),
 ('bufón', 0.30569782853126526),
 ('herrero', 0.30319517850875854),
 ('ante', 0.29895052313804626),
 ('rápidamente', 0.27513277530670166),
 ('puente', 0.2616966664791107),
 ('merlín', 0.2594411373138428),
 ('mucho', 0.2577259838581085)]

In [47]:
w2v_model_2.wv.most_similar(positive=["armadura"], topn=15)

[('después', 0.359968364238739),
 ('cima', 0.3396948575973511),
 ('vida', 0.33104708790779114),
 ('pensar', 0.3214288651943207),
 ('bueno', 0.319081574678421),
 ('visera', 0.30767378211021423),
 ('hijo', 0.29840072989463806),
 ('puerta', 0.29037338495254517),
 ('barba', 0.28004884719848633),
 ('espada', 0.2749238610267639),
 ('habitación', 0.2736709713935852),
 ('fuego', 0.27049967646598816),
 ('hombro', 0.2695312798023224),
 ('misma', 0.2670157551765442),
 ('única', 0.26016175746917725)]

In [48]:
w2v_model_2.wv.most_similar(positive=["julieta"], topn=15)

[('amaba', 0.3950004279613495),
 ('fue', 0.3402339518070221),
 ('habían', 0.31799033284187317),
 ('parte', 0.3163617253303528),
 ('necesitado', 0.31185752153396606),
 ('hablar', 0.31131711602211),
 ('aquello', 0.2874036729335785),
 ('amado', 0.28302252292633057),
 ('ese', 0.2779780328273773),
 ('nota', 0.275251179933548),
 ('poco', 0.27507615089416504),
 ('bueno', 0.2705758512020111),
 ('perdido', 0.2701999545097351),
 ('les', 0.2662353217601776),
 ('había', 0.2624650299549103)]

In [49]:
w2v_model_2.wv.most_similar(positive=["castillo"], topn=10)

[('espejo', 0.2969150245189667),
 ('continuó', 0.28406602144241333),
 ('entró', 0.2702958583831787),
 ('pensamiento', 0.26796311140060425),
 ('bufón', 0.2673908472061157),
 ('eres', 0.2555221915245056),
 ('hombre', 0.2526976466178894),
 ('poco', 0.2521313428878784),
 ('silencio', 0.24583519995212555),
 ('miedo', 0.24568074941635132)]

e) Palabras que MENOS se parecen:

In [56]:
# Palabras que MENOS se relacionan con...:
w2v_model_2.wv.most_similar(negative=["armadura"], topn=10)

[('la', 0.3934031128883362),
 ('tendréis', 0.3143630027770996),
 ('del', 0.307258278131485),
 ('abrir', 0.2981376647949219),
 ('mucho', 0.28400400280952454),
 ('mientras', 0.2770874500274658),
 ('dijo', 0.26948556303977966),
 ('dragones', 0.2631210386753082),
 ('puente', 0.24549484252929688),
 ('desapareció', 0.24121452867984772)]

In [50]:
# Palabras que MENOS se relacionan con...:
w2v_model_2.wv.most_similar(negative=["caballero"], topn=10)

[('hace', 0.35617825388908386),
 ('estos', 0.35594308376312256),
 ('ayuda', 0.3500673174858093),
 ('medida', 0.3406900465488434),
 ('cruzada', 0.3170248568058014),
 ('tanto', 0.31408166885375977),
 ('viaje', 0.3133825361728668),
 ('los', 0.30042967200279236),
 ('tener', 0.2946837544441223),
 ('difícil', 0.2939636707305908)]

In [51]:
# Palabras que MENOS se relacionan con...:
w2v_model_2.wv.most_similar(negative=["julieta"], topn=10)

[('del', 0.3303414583206177),
 ('debe', 0.32508790493011475),
 ('hay', 0.3218531906604767),
 ('tenéis', 0.32151517271995544),
 ('¿qué', 0.310785174369812),
 ('explicó', 0.28930947184562683),
 ('ir', 0.2835199236869812),
 ('espejo', 0.27972033619880676),
 ('puedes', 0.2757706344127655),
 ('puedo', 0.2705800533294678)]

In [52]:
w2v_model_2.wv.most_similar(negative=["ardilla"], topn=15)

[('quitarse', 0.4007602632045746),
 ('sabía', 0.38899457454681396),
 ('de', 0.3342830240726471),
 ('enorme', 0.30914440751075745),
 ('ponerse', 0.30849120020866394),
 ('dentro', 0.3062536418437958),
 ('atrapado', 0.29749417304992676),
 ('todas', 0.2820086181163788),
 ('mismo', 0.2789800465106964),
 ('reino', 0.27278584241867065),
 ('cruzada', 0.26729750633239746),
 ('era', 0.2666422724723816),
 ('parecía', 0.266150563955307),
 ('porque', 0.2587030231952667),
 ('dragones', 0.2536733150482178)]

In [53]:
w2v_model_2.wv.most_similar(negative=["dragón"], topn=15)

[('durante', 0.4075787663459778),
 ('hablando', 0.34886059165000916),
 ('bien', 0.34122148156166077),
 ('sido', 0.3111303448677063),
 ('exclamó', 0.30669480562210083),
 ('no', 0.2982860207557678),
 ('verdadero', 0.2856340706348419),
 ('soy', 0.2825816571712494),
 ('dicho', 0.27722615003585815),
 ('tanto', 0.267424613237381),
 ('tendréis', 0.26650315523147583),
 ('misma', 0.2597801685333252),
 ('hermoso', 0.25772494077682495),
 ('realidad', 0.2555617094039917),
 ('podido', 0.25137418508529663)]

f) Gráfico con TSNE

In [54]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

x_vals, y_vals, labels = reduce_dimensions(w2v_model_2)

MAX_WORDS=200
fig = px.scatter(x=x_vals[:MAX_WORDS], y=y_vals[:MAX_WORDS], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:783: FutureWarning:

The default initialization in TSNE will change from 'random' to 'pca' in 1.2.

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning:

The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.



### Conclusiones:

El texto utilizado para crear los embeddings es un cuento en español llamado "El caballero de la armadura oxidada". El corpus resultante contiene 1121 documentos y un vocabulario de 455 palabras. Utilicé librería Gensim y probé con ambos modelos Skipgram y CBOW. En los dos casos, consideré:
  *  Dimensión de los vectores de salida: 50
  *  Mínimo numero de veces que tiene que estar repetida la palabra para quedar dentro del vocabulario: 5
  *  Cantidad de palabras antes y después de la palabra target: 2
  *  Entrenamiento con 2000 epochs.
Para determinar el vocabulario, usé el tokenizador de Keras y TSNE de sklearn para graficar los resultados tal como hicimos en clase.


Observaciones


*   El tokenizador de Keras elimina las mayúsculas, por lo cual hay que tener esto en cuenta cuando se hace la búsqueda de palabras similares.
*   Como aquí no se utiliza stemming, se observa que el vocabulario aparecen palabras con la misma raíz (ejemplos: "amor" y "amoroso", "caballero" y "caballeros").
*   En el cuento habían muchos diálogos, por lo cual fue necesario preprocesar el texto para quitar quitar caracteres como guiones, ya que Gensim no los elimina y al realizar los ensayos el modelo detectaba estos elementos como los más parecidos a los nombres de los personajes principales (el caballero o Merlín, por ejemplo).
*   Los modelos parecen funcionar bastante bien a pesar que la relación entre documentos y vocabulario no es tan significativa (nro de documentos aprox 2,5 veces más grande que nro de palabras en el vocabulario).
*   Como no se removieron las stop-words, estas aparecen muy asociadas a varias de las palabras analizadas - por ejemplo para la palabra "caballero" el modelo encuentra el artículo "el" como la palabra más parecida (similitud coseno = 0.91 con Skipgram).

*   Similitudes:
      * Cuando se busca "caballero", los dos algoritmos encuentran que "el" es la palabra más parecida. CBOW parece dar un mejor resultado porque encuentra palabras como "rey", "mago", "Merlín y "dragón" que corresponden a personajes fuertemente asociados con el caballero protagonista.
      * Cuando se busca "armadura", Skipgram encuentra por ejemplo "lado", "realidad", "desconocido", "aprender" y CBOW en cambio "barba", "vida", "cima" y "visera". Los resultados son considerablemente distintos pero en ambos casos tienen relación con la armadura.
      * Cuando se busca "julieta" los dos modelos encuentran "amaba" y "amado" que es correcto ya es que la esposa del caballero. 
      * Si se busca "castillo" , Skipgram devuelve "silencio" y "conocimiento" como más parecidas, lo que tiene mucho sentido en relación con la temática de los capítulos 4 ("El castillo del silencio") y 5 ("El castillo del conocimiento").

*  Diferencias: Cuando se busca las palabras menos parecidas a "armadura" por ejemplo, el modelo devuelve algunas opciones como "puente", "levadizo", "espejo" (Skipgram) y dragones (CBOW). Esto tiene mucho sentido porque el dragon y el puente levadizo aparecen hacia el final de la historia (capítulo 6), donde el caballero ya se ha librado de la mayor parte de su armadura y por lo tanto ya casi no se menciona.

* Gráficos de TSNE: El gráfico para el modelo que usa Skipgram parece estar mucho más concentrado en el centro. Haciendo zoom, se observan varios grupos de palabras interesantes: "caballero", "armadura" y "atrapado" que representan el tema central del cuento, y por otro lado "rebeca", "ardilla", "animales", "miedo" y "hablar" que resumen el viaje del caballero a lo largo de los diferentes castillos acompañado de la paloma rebeca y la ardilla. 
CBOW muestra resultados similares, pero junto a la palabra "caballero" coloca también a "julieta" su esposa, que se menciona repetidamente a lo largo de la historia, y "herrero", que es a quien recurre el caballero en el primer capítulo para quitarse la armadura.













